In [19]:
import json
import re
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.webdriver import WebDriver
from webdriver_manager.chrome import ChromeDriverManager

# Function to get the authenticated session
def get_authenticated_session():
    # Replace this function with the code to log in and obtain an authenticated session
    # For example, you might use Selenium to log in and get the session cookies.
    # Here, I'm providing a simple example using the requests library to create a session.
    session = requests.session()
    session.headers["User-Agent"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    return session

# Function to create a Selenium session
def create_authenticated_session():
    # Set up ChromeOptions
    chrome_options = Options()
    chrome_options.add_argument("--user-data-dir=C:/Users/Bas van Houten/AppData/Local/Google/Chrome/User Data/Profile 4")
    chrome_options.add_argument("--headless") 

    # Set up ChromeDriverManager to automatically download and manage the ChromeDriver executable
    driver = WebDriver(service=Service(ChromeDriverManager().install()), options=chrome_options)

    return driver

# Use regular expressions to extract JSON data from script tags
json_pattern = re.compile(r"var graphData = (\[.*?\]);", re.DOTALL)

# Step 1: Get the authenticated session
session = get_authenticated_session()

# Step 2: Create the Selenium session using the function we defined earlier
driver = create_authenticated_session()

# Step 3: Use the authenticated session to make subsequent requests with Selenium
url = "https://pro.imdb.com/title/tt3564472"  # Replace with the URL you want to scrape
driver.get(url)

# Wait for the page to load, you may need to adjust the time depending on the website
driver.implicitly_wait(5)

# Get the page source after waiting for the content to load
page_source = driver.page_source

# Find the script tag containing the graph data using regular expressions
match = json_pattern.search(page_source)

if match:
    json_data = match.group(1)
    graph_data = json.loads(json_data)

    # Now, graph_data contains a list of dictionaries, each containing "date" and "rank" information
    for data in graph_data:
        date = data.get("date")
        rank = data.get("rank")
        print("Date:", date)
        print("Rank:", rank)
        print("-----------")

# Step 4: Don't forget to close the driver when you are done
driver.quit()


Date: 2014-03-02T00:00:00.000Z
Rank: 282889
-----------
Date: 2014-03-09T00:00:00.000Z
Rank: 89258
-----------
Date: 2014-03-16T00:00:00.000Z
Rank: 86732
-----------
Date: 2014-03-23T00:00:00.000Z
Rank: 61367
-----------
Date: 2014-03-30T00:00:00.000Z
Rank: 107131
-----------
Date: 2014-04-06T00:00:00.000Z
Rank: 102759
-----------
Date: 2014-04-13T00:00:00.000Z
Rank: 126998
-----------
Date: 2014-04-20T00:00:00.000Z
Rank: 86391
-----------
Date: 2014-04-27T00:00:00.000Z
Rank: 110227
-----------
Date: 2014-05-04T00:00:00.000Z
Rank: 104965
-----------
Date: 2014-05-11T00:00:00.000Z
Rank: 124206
-----------
Date: 2014-05-18T00:00:00.000Z
Rank: 131526
-----------
Date: 2014-06-01T00:00:00.000Z
Rank: 133212
-----------
Date: 2014-06-08T00:00:00.000Z
Rank: 124796
-----------
Date: 2014-06-15T00:00:00.000Z
Rank: 130431
-----------
Date: 2014-06-22T00:00:00.000Z
Rank: 127679
-----------
Date: 2014-06-29T00:00:00.000Z
Rank: 88589
-----------
Date: 2014-07-06T00:00:00.000Z
Rank: 116719
---------

In [ ]:
## save point 

# This is the code we want

In [60]:
import json
import re
import csv
import pandas as pd  # Import the pandas library
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.webdriver import WebDriver
from webdriver_manager.chrome import ChromeDriverManager
import os
import requests


# Function to get the authenticated session
def get_authenticated_session():
    # Replace this function with the code to log in and obtain an authenticated session
    # For example, you might use Selenium to log in and get the session cookies.
    # Here, I'm providing a simple example using the requests library to create a session.
    session = requests.session()
    session.headers["User-Agent"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    return session

# Function to create a Selenium session
def create_authenticated_session():
    # Set up ChromeOptions
    chrome_options = Options()
    chrome_options.add_argument("--user-data-dir=C:/Users/Bas van Houten/AppData/Local/Google/Chrome/User Data/Profile 4")
    chrome_options.add_argument("--headless") 

    # Set up ChromeDriverManager to automatically download and manage the ChromeDriver executable
    driver = WebDriver(service=Service(ChromeDriverManager().install()), options=chrome_options)

    return driver

# Use regular expressions to extract JSON data from script tags
json_pattern = re.compile(r"var graphData = (\[.*?\]);", re.DOTALL)

# List of movie identifiers
# Load the dataset from the CSV file using pandas
dataset_path = "C:/Users/Bas van Houten/Documents/Master/Thesis/gen/data-preparation/temp/main_base.csv"
data_frame = pd.read_csv(dataset_path)

# Extract the IMDb IDs from the "imdb_id" column of the dataset
movie_identifiers = data_frame["imdb_id"].tolist()

# Only scrape the first 3 movie identifiers
movie_identifiers = movie_identifiers[27:100]

# File to store scraped movie identifiers
scraped_movie_file = "scraped_movies.txt"

# Read the scraped movie identifiers from the file
scraped_movie_identifiers = set()
if os.path.exists(scraped_movie_file):
    with open(scraped_movie_file, "r") as file:
        scraped_movie_identifiers = set(file.read().splitlines())

# Step 1: Get the authenticated session
session = get_authenticated_session()

# Step 2: Create the Selenium session using the function we defined earlier
driver = create_authenticated_session()

# Get all the dates in weekly intervals from February 1st, 1998, to today
start_date = datetime(1998, 2, 1)
end_date = datetime.now()
all_dates = []
current_date = end_date

while current_date >= start_date:
    all_dates.append(current_date.strftime("%Y-%m-%d"))
    current_date -= timedelta(weeks=1)

# Step 3: Collect the data for each movie and date
movie_data_by_date = {}

# Initialize the movie_data_by_date dictionary with all_dates
for movie_id in movie_identifiers:
    movie_data_by_date[movie_id] = {date: "N/A" for date in all_dates}

# Update the set of scraped movie identifiers
updated_scraped_movie_identifiers = set()

for movie_id in movie_identifiers:
    url = f"https://pro.imdb.com/title/{movie_id}"
    driver.get(url)

    # Get the page source
    page_source = driver.page_source

    # Find the script tag containing the graph data using regular expressions
    match = json_pattern.search(page_source)

    if match:
        json_data = match.group(1)
        graph_data = json.loads(json_data)

        for data in graph_data:
            date = data.get("date")
            rank = data.get("rank")

            # Convert the date format from "2014-03-02T00:00:00.000Z" to "2014-03-02"
            date = date[:10]

            # Find the closest weekly date for the current data entry
            closest_date = min(all_dates, key=lambda x: abs(datetime.strptime(x, "%Y-%m-%d") - datetime.strptime(date, "%Y-%m-%d")))

            # Add the rank for the movie_id and date
            movie_data_by_date[movie_id][closest_date] = rank

        # Add the scraped movie identifier to the set
        updated_scraped_movie_identifiers.add(movie_id)

# Step 4: Save the data in a JSON file
with open("movie_ranks.json", "a") as jsonfile:  # Use "a" mode for append
    json.dump(movie_data_by_date, jsonfile)

# Step 5: Save the updated set of scraped movie identifiers to the file (append mode)
with open(scraped_movie_file, "a") as file:  # Use "a" mode for append
    file.write("\n".join(updated_scraped_movie_identifiers))  # Append the new identifiers to the existing file

# Step 6: Don't forget to close the driver when you are done
driver.quit()